<a href="https://colab.research.google.com/github/joangog/object-detection/blob/main/coco2017_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model comparison on COCO 2017 dataset
*   SSD300 VGG16
*   Faster R-CNN ResNet-50 FPN
*   Mask R-CNN ResNet-50 FPN
*   YOLOv5s
*   YOLOv5m






### Get requirements

In [ ]:
%%shell

# Install Yolov5
cd /content
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install --quiet -r requirements.txt

In [ ]:
%%shell

# Clone torchvision repository and get aux files
cd /content
git clone https://github.com/pytorch/vision.git
cd vision
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

### Import packages

In [13]:
# Import packages

import os
from google.colab import files

import math
import sys
import time
import numpy as np
import json
import cv2

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision.models.detection import ssd300_vgg16, fasterrcnn_resnet50_fpn, maskrcnn_resnet50_fpn
from torchvision.datasets import CocoDetection
from torchvision.transforms import Compose, Resize, ToTensor

import utils
from utils import collate_fn
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator

### Define aux functions

In [14]:
# Copied from repo vision/references/detection/engine.py
def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

# Copied from repo vision/references/detection/engine.py with adjustments
@torch.no_grad()
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)
        targets = [sample for target in targets for sample in target]  # Added this line

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"]: output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator


### Download COCO 2017 dataset

In [ ]:
%%shell

# Download images
cd /content
wget 'http://images.cocodataset.org/zips/val2017.zip'
unzip -q 'val2017.zip'
rm 'val2017.zip'

In [ ]:
%%shell

# Download annotations
cd /content
wget 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
unzip -q 'annotations_trainval2017.zip'
rm 'annotations_trainval2017.zip'
cp '/content/annotations/instances_val2017.json' '/content/val2017'

### Load COCO 2017 dataset

In [4]:
img_dir = '/content/val2017'
ann_file = os.path.join(img_dir,'instances_val2017.json')  # annotations

# Define data transforms
transforms = Compose([Resize((640,640)), ToTensor()])

# Create dataset
dataset = CocoDetection(img_dir, ann_file, transform = transforms)

# # Create data loader
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


### Load pre-trained model

In [5]:
# @markdown Model Selection { display-mode: 'form', run: 'auto' }
model_name = 'SSD300 VGG16' # @param ['SSD300 VGG16', 'Faster R-CNN ResNet-50 FPN', 'Mask R-CNN ResNet-50 FPN', 'YOLOv5s', 'YOLOv5m']

if model_name == 'SSD300 VGG16':
  model_id = 'ssd'
  model = ssd300_vgg16(pretrained=True, progress=True)
elif model_name == 'Faster R-CNN ResNet-50 FPN':
  model_id = 'fasterrcnn'
  model = fasterrcnn_resnet50_fpn(pretrained=True, progress=True)
elif model_name == 'Mask R-CNN ResNet-50 FPN':
  model_id = 'maskrcnn'
  model = maskrcnn_resnet50_fpn(pretrained=True, progress=True)
elif model_name == 'YOLOv5s':
  model_id = 'yolov5s'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
elif model_name == 'YOLOv5m':
  model_id = 'yolov5m'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

print('Loaded model: '+ model_name + '\n')


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loaded model: SSD300 VGG16



### Evaluate model

In [ ]:
# Get appropriate device for model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Evaluate model
evaluate(model, data_loader, device) 


/content
Test:  [   0/2500]  eta: 0:18:54  model_time: 0.2524 (0.2524)  evaluator_time: 0.0101 (0.0101)  time: 0.4539  data: 0.1876  max mem: 240
Test:  [ 100/2500]  eta: 0:08:18  model_time: 0.1766 (0.1779)  evaluator_time: 0.0130 (0.0205)  time: 0.2010  data: 0.0049  max mem: 240
Test:  [ 200/2500]  eta: 0:07:49  model_time: 0.1762 (0.1766)  evaluator_time: 0.0133 (0.0194)  time: 0.1977  data: 0.0039  max mem: 240
Test:  [ 300/2500]  eta: 0:07:25  model_time: 0.1751 (0.1760)  evaluator_time: 0.0135 (0.0187)  time: 0.1998  data: 0.0033  max mem: 240
Test:  [ 400/2500]  eta: 0:07:05  model_time: 0.1753 (0.1760)  evaluator_time: 0.0108 (0.0192)  time: 0.1974  data: 0.0036  max mem: 240
Test:  [ 500/2500]  eta: 0:06:44  model_time: 0.1748 (0.1757)  evaluator_time: 0.0122 (0.0191)  time: 0.1986  data: 0.0040  max mem: 241
Test:  [ 600/2500]  eta: 0:06:23  model_time: 0.1745 (0.1755)  evaluator_time: 0.0141 (0.0190)  time: 0.2011  data: 0.0041  max mem: 241
Test:  [ 700/2500]  eta: 0:06:04

## Save results

In [ ]:
results = []
for i, img_id in enumerate(img_ids):  # For every image
  img_predictions = predictions[i]
  pred_boxes = img_predictions['boxes'].tolist()
  pred_labels = img_predictions['labels'].tolist()
  pred_scores = img_predictions['scores'].tolist()
  for j in range(0,len(pred_box)):  # For every predicted object
    pred_box = pred_boxes[j]
    results.append({
        'image_id': img_id,
        'category_id': pred_labels[j],
        'bbox': [round(val,1) for val in pred_box],  # Round for lower file size
        'score': pred_scores[j]
        })

results_file = os.path.join(img_dir,f'{model_id}_coco17_results.json')
with open(results_file, 'w') as outfile: 
    json.dump(results, outfile,indent = 2)



In [ ]:
files.download(results_file) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# results_dataset = dataset.loadRes(results_file)
# eval = cocoeval.COCOeval(dataset, results_dataset, iouType='bbox')
# eval.evaluate()
# eval.accumulate()
# eval.summarize()

